In [ ]:
# to automatically reload modules who's content has changed
%load_ext autoreload
%autoreload 2

# configure matplotlib
%matplotlib inline
#%config InlineBackend.figure_format = 'svg'

In [ ]:
import time
import numpy as np
import GPy
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()

import ipywidgets as widgets
from IPython.display import display
from IPython.core.debugger import set_trace

In [ ]:
import function_bo as fbo
from function_bo_plotting import *

import sys
sys.path.append('../../spring_task')
from pygame_spring import Simulation

In [ ]:
sim = Simulation()

In [ ]:
def plot_summary(states):
    fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(20, 15))
    ts = [s.t for s in states]
    
    ax1.set_title('positions')
    ax1.plot(ts, [s.target.y for s in states], label='target y')
    ax1.plot(ts, [s.ee_pos.y for s in states], label='EE y')
    ax1.set_ylabel('y position')
    ax1.set_xlabel('time')
    ax1.legend()
    
    ax2.set_title('recorded gains')
    ax2.plot(ts, [s.Kp for s in states], label='Kp')
    ax2.plot(ts, [s.Kd for s in states], label='Kp')
    ax2.set_xlabel('time')
    ax2.legend()
    
    ax3.set_title('position error')
    ax3.plot(ts, [s.target.y-s.ee_pos.y for s in states], label='error')
    ax3.set_xlabel('time')
    ax3.set_ylabel('error')
    ax3.legend()
    
    ax4.set_title('force')
    ax4.plot(ts, [s.spring_f.y for s in states], label='real spring force y')
    ax4.set_xlabel('time')
    ax4.set_ylabel('y force')
    ax4.legend()


In [ ]:
def sigmoid(x):
    return 1 / (1 + np.exp(-x))



def trajectory(t):
    if t < 6:
        return (0, 100 + 50*t)
    else:
        return None # finish
    
f_max = 600
Kp_max = 6

def objective(f, fps=2000):
    def normalize_spring_f(spring_f_y):
        return np.abs(spring_f_y)/f_max
        
    def gains(spring_f):
        Kp_y = f(normalize_spring_f(spring_f.y)) * Kp_max
        return (Kp_y, 0)
    states = sim.run(fps=fps, virtual_spring_gains=gains, real_spring_gains=(1.5, 0.2), ee_initial_pos=(0, 50), trajectory=trajectory)
    R_ls = []
    R_g = 0
    eval_info = {'states':states}
    
    n = len(states)
    penalty = []
    closeness = []
    
    for i, s in enumerate(states):
        reaction_f = -1 * s.spring_f
        penalty.append(np.linalg.norm(np.multiply(reaction_f, s.ee_vel)))
        closeness.append(np.linalg.norm(s.target-s.ee_pos))
    
    # np.diff takes the differences between adjacent elements
    force_reward = 0.5*sigmoid(np.array(penalty)) + 0.5*sigmoid(np.hstack([np.diff(penalty), 0]))
    goal_reward = 1.5*sigmoid(np.array(closeness))
    reward = -1 * force_reward + goal_reward
    
    #sample_at = np.random.permutation(np.arange(len(states)))[:15]
    sample_at = np.arange(len(states))
    R_ls = [(normalize_spring_f(states[i].spring_f.y), reward[i]) for i in sample_at]
    R_g = np.sum(reward)

    return R_ls, R_g, eval_info

In [ ]:
def test_constant_gains():
    R_ls, R_g, eval_info = objective(lambda f: 1)
    s = eval_info['states']
    print(R_g)
    print(R_ls)
    plot_summary(s)
test_constant_gains()

In [ ]:
domain_bounds = ('f', 0, 1) # spring force
range_bounds = (0.1, 1)   # stiffness

class Coordinator(fbo.Coordinator):
    def get_pre_phase_config(self, trial_num):
        c = fbo.GPPriorSelectConfig(self.domain_bounds)
        c.mu = lambda x: 2.5 + x
        #c.kernel = GPy.kern.Bias(input_dim=1, variance=0.2) + GPy.kern.Linear(input_dim=1, variances=(1,))
        c.kernel = GPy.kern.RBF(input_dim=1, variance=2, lengthscale=1)
        return c

    def get_bayes_config(self, trial_num):
        c = fbo.BayesSelectConfig(self.domain_bounds)
        c.kernel = GPy.kern.RBF(input_dim=2, ARD=False)
        #c.sparse_GP = True
        #c.surrogate_optimise_iterations = 2
        #c.surrogate_optimise_parallel = False # can't with sparse GP
        #c.surrogate_optimise_verbose = True
        #c.tracking_l = 0.4
        return c
    
coordinator = Coordinator(domain_bounds, 10, 14)

In [ ]:
np.random.seed(0)
op = fbo.Optimiser(objective, domain_bounds, range_bounds, desired_extremum='min', coordinator=coordinator)
op.run()
plot_convergence(op, best_R_g=None)
plot_trials(op, op.trials, color_by_reward=True)
plot_surrogate_with_trials(op, -1)

inc_i, inc = op.get_incumbent()
print('incumbent = trial {}'.format(inc_i))
plot_trials(op, [inc], color_by_reward=True)
#plot_trial_area(op, inc, to_fit)

In [ ]:
plot_surrogate_3D(op, op.trials[-1].surrogate)

In [ ]:
def test_incumbent():
    R_ls, R_g, eval_info = objective(inc.f, fps=20)
    s = eval_info['states']
    print(R_g)
    print(R_ls)
    plot_summary(s)
test_incumbent()